In [3]:
# pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   --------------- ------------------------ 4.2/10.6 MB 36.1 MB/s eta 0:00:01
   ----------------------------------- ---- 9.4/10.6 MB 30.9 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 27.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pygame
import random
import math

pygame.init()

# ============================================
# คลาส LavaPit - บ่อลาวา (อุปสรรค)
# ============================================
class LavaPit:
    """บ่อลาวาที่จะทำให้สูญเสียคน"""
    def __init__(self, x, z):
        self.x = x  # 0.0 - 1.0 (ซ้าย-ขวา)
        self.z = z  # 0.0 - 1.0 (ไกล-ใกล้)
        self.base_width = 80
        self.base_height = 50
        self.damage = random.randint(5, 15)
        self.active = True
        self.color = (255, 165, 0)  # ORANGE
        
    def draw(self):
        """วาดบ่อลาวา"""
        if self.active:
            screen_x, screen_y, scale = world_to_screen(self.x, self.z)
            width = int(self.base_width * scale)
            height = int(self.base_height * scale)
            
            offset_x = random.randint(-2, 2)
            offset_y = random.randint(-2, 2)
            pygame.draw.ellipse(screen, self.color, 
                              (int(screen_x - width//2 + offset_x), 
                               int(screen_y - height//2 + offset_y), 
                               width, height))
            pygame.draw.ellipse(screen, RED, 
                              (int(screen_x - width//2 + offset_x + 5), 
                               int(screen_y - height//2 + offset_y + 5), 
                               max(10, width - 10), max(10, height - 10)))
    
    def check_collision(self, crowd):
        """เช็คชนกับฝูงชน"""
        if not self.active or len(crowd.people) == 0:
            return False
        
        hit = False
        for person in crowd.people[:]:
            dist_x = abs(person.x - self.x)
            dist_z = abs(person.z - self.z)
            
            if dist_x < 0.08 and dist_z < 0.08:
                hit = True
                break
        
        if hit:
            crowd.remove_people(min(self.damage, crowd.count))
            self.active = False
            return True
        return False


# ============================================
# ตั้งค่าเกม
# ============================================
WIDTH, HEIGHT = 800, 800 
FPS = 60

# ความเร็ว (ลดลงให้เล่นง่ายขึ้น)
SCROLL_SPEED = 4
PLAYER_SPEED = 13 
BULLET_SPEED = 15
ENEMY_SPEED = 1.5  

# ขอบเขตถนน (3D Perspective)
# ด้านหลัง (ไกล) - แคบ
ROAD_TOP_LEFT = WIDTH // 2 - 80
ROAD_TOP_RIGHT = WIDTH // 2 + 80
ROAD_TOP_Y = 50

# ด้านหน้า (ใกล้) - กว้าง
ROAD_BOTTOM_LEFT = 50
ROAD_BOTTOM_RIGHT = WIDTH - 50
ROAD_BOTTOM_Y = HEIGHT - 80  # ปรับตามความสูงใหม่

# ฟังก์ชันแปลงพิกัด 3D เป็น 2D (perspective projection)
def world_to_screen(x, z):
    """
    แปลงพิกัด world (x: ซ้าย-ขวา, z: ไกล-ใกล้) เป็นพิกัดหน้าจอ
    z = 0 (ไกลสุด ด้านหลัง) ถึง z = 1 (ใกล้สุด ด้านหน้า)
    """
    # คำนวณตำแหน่ง x บนหน้าจอตาม perspective
    screen_left = ROAD_TOP_LEFT + (ROAD_BOTTOM_LEFT - ROAD_TOP_LEFT) * z
    screen_right = ROAD_TOP_RIGHT + (ROAD_BOTTOM_RIGHT - ROAD_TOP_RIGHT) * z
    screen_x = screen_left + (screen_right - screen_left) * x
    
    # คำนวณตำแหน่ง y บนหน้าจอ
    screen_y = ROAD_TOP_Y + (ROAD_BOTTOM_Y - ROAD_TOP_Y) * z
    
    # คำนวณขนาดตาม perspective (ไกล = เล็ก, ใกล้ = ใหญ่)
    scale = 0.3 + 0.7 * z  # scale จาก 0.3 (ไกล) ถึง 1.0 (ใกล้)
    
    return screen_x, screen_y, scale

# ฟังก์ชันคำนวณความกว้างถนนที่ z ต่างๆ
def get_road_bounds(z):
    """คืนค่าขอบซ้าย-ขวาของถนนที่ระดับความลึก z"""
    left = ROAD_TOP_LEFT + (ROAD_BOTTOM_LEFT - ROAD_TOP_LEFT) * z
    right = ROAD_TOP_RIGHT + (ROAD_BOTTOM_RIGHT - ROAD_TOP_RIGHT) * z
    return left, right

# สี
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 150, 255)
CYAN = (0, 200, 255)
RED = (255, 50, 50)
GREEN = (50, 255, 100)
YELLOW = (255, 255, 100)
ORANGE = (255, 165, 0)
GRAY = (40, 40, 40)
DARK_GRAY = (30, 30, 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Math Crowd Runner")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 48)
small_font = pygame.font.Font(None, 28)
big_font = pygame.font.Font(None, 72)
huge_font = pygame.font.Font(None, 36)

# โหลดรูปภาพ
try:
    ai_hoshino_img = pygame.image.load('ai_hoshino.jpg')
    ai_hoshino_img = pygame.transform.scale(ai_hoshino_img, (30, 30))
except:
    ai_hoshino_img = None

try:
    knife_img = pygame.image.load('knife.png')
    knife_img = pygame.transform.scale(knife_img, (20, 20))
except:
    knife_img = None

try:
    stage_bg = pygame.image.load('stage.png')
    stage_bg = pygame.transform.scale(stage_bg, (WIDTH, HEIGHT))
except:
    stage_bg = None

try:
    ai_dead_img = pygame.image.load('ai_dead.jpg')
    ai_dead_img = pygame.transform.scale(ai_dead_img, (WIDTH, HEIGHT))  # เต็มจอ
except:
    ai_dead_img = None

try:
    ai_win_img = pygame.image.load('ai_win.png')
    ai_win_img = pygame.transform.scale(ai_win_img, (WIDTH, HEIGHT))
except:
    ai_win_img = None

# โหลดเพลง
try:
    pygame.mixer.music.load('bg_song.mp3')
except:
    pass

# Game State
game_state = "MENU"
current_level = 1


# ============================================
# คลาส Operation - การดำเนินการทางคณิตศาสตร์
# ============================================
class Operation:
    """คลาสสำหรับเก็บการดำเนินการทางคณิตศาสตร์"""
    def __init__(self, symbol, func):
        self.symbol = symbol  # สัญลักษณ์ที่แสดงบน gate เช่น '+5', 'x2', '√'
        self.func = func      # ฟังก์ชันที่จะนำไปใช้กับจำนวนคน
    
    def apply(self, crowd):
        """ใช้ effect กับ crowd"""
        self.func(crowd)


# ============================================
# Operations Pool - ชุดการดำเนินการแต่ละระดับ
# ============================================

# Level 1: บวก ลบ คูณ พื้นฐาน
ops_level_1 = [
    Operation('+10', lambda c: c.add_people(10)),
    Operation('+20', lambda c: c.add_people(20)),
    Operation('+30', lambda c: c.add_people(30)),
    Operation('+50', lambda c: c.add_people(50)),
    Operation('-5', lambda c: c.remove_people(5)),
    Operation('-10', lambda c: c.remove_people(10)),
    Operation('-20', lambda c: c.remove_people(20)),
    Operation('-30', lambda c: c.remove_people(30)),
    Operation('-50', lambda c: c.remove_people(50)),
    Operation('x2', lambda c: c.multiply_people(2)),
    Operation('x3', lambda c: c.multiply_people(3)),
]

# Level 2: เพิ่ม หาร รากที่สอง ยกกำลัง
ops_level_2 = [
    Operation('x2', lambda c: c.multiply_people(2)),
    Operation('x3', lambda c: c.multiply_people(3)),
    Operation('x4', lambda c: c.multiply_people(4)),
    Operation('x5', lambda c: c.multiply_people(5)),
    Operation('÷2', lambda c: c.divide_people(2)),
    Operation('÷3', lambda c: c.divide_people(3)),
    Operation('÷4', lambda c: c.divide_people(4)),
    Operation('÷5', lambda c: c.divide_people(5)),
    Operation('÷6', lambda c: c.divide_people(6)),
    Operation('÷7', lambda c: c.divide_people(7)),
    Operation('÷8', lambda c: c.divide_people(8)),
    Operation('÷10', lambda c: c.divide_people(10)),
    Operation('√', lambda c: c.sqrt_people()),
    Operation('^2', lambda c: c.power_people(2)),
    Operation('^3', lambda c: c.power_people(3)),
    Operation('+30', lambda c: c.add_people(30)),
    Operation('-30', lambda c: c.remove_people(30)),
]

# Level 3: ยากสุด
ops_level_3 = [
    Operation('x2', lambda c: c.multiply_people(2)),
    Operation('x3', lambda c: c.multiply_people(3)),
    Operation('x4', lambda c: c.multiply_people(4)),
    Operation('x5', lambda c: c.multiply_people(5)),
    Operation('÷2', lambda c: c.divide_people(2)),
    Operation('÷3', lambda c: c.divide_people(3)),
    Operation('÷5', lambda c: c.divide_people(5)),
    Operation('÷7', lambda c: c.divide_people(7)),
    Operation('÷10', lambda c: c.divide_people(10)),
    Operation('÷15', lambda c: c.divide_people(15)),
    Operation('÷20', lambda c: c.divide_people(20)),
    Operation('√', lambda c: c.sqrt_people()),
    Operation('^2', lambda c: c.power_people(2)),
    Operation('^3', lambda c: c.power_people(3)),
    Operation('+40', lambda c: c.add_people(40)),
    Operation('-40', lambda c: c.remove_people(40)),
]


# ============================================
# คลาส Button
# ============================================
class Button:
    def __init__(self, x, y, width, height, text, color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover = False
        
    def draw(self):
        color = self.color if not self.hover else tuple(min(c + 30, 255) for c in self.color)
        pygame.draw.rect(screen, color, self.rect)
        pygame.draw.rect(screen, WHITE, self.rect, 3)
        
        text_surf = font.render(self.text, True, WHITE)
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)
    
    def check_click(self, pos):
        self.hover = self.rect.collidepoint(pos)
        return self.hover


# ============================================
# คลาส Person - คนในทีมเรา
# ============================================
class Person:
    """คนหนึ่งคนในทีม มีปืน"""
    def __init__(self, x, z, crowd_size=10):
        self.x = x  # 0.0 - 1.0 (ซ้าย-ขวา)
        self.z = z  # 0.0 - 1.0 (ไกล-ใกล้)
        self.base_size = 10
        self.crowd_size = crowd_size
        self.update_size()
        self.color = BLUE
        self.has_gun = True
        self.shoot_cooldown = 0
        self.shoot_rate = 20
    
    def update_size(self):
        """ปรับขนาดตามจำนวนคนในทีม"""
        if self.crowd_size <= 10:
            self.base_size = 15
        elif self.crowd_size <= 50:
            self.base_size = 12
        elif self.crowd_size <= 100:
            self.base_size = 10
        else:
            self.base_size = max(8, 15 - int(self.crowd_size / 50))
        
    def update(self, target_x, target_z):
        """เคลื่อนที่ไปหาตำแหน่งในฟอร์เมชั่น"""
        dx = target_x - self.x
        dz = target_z - self.z
        dist = math.sqrt(dx*dx + dz*dz)
        
        if dist > 0.01:
            speed = 0.02
            self.x += (dx / dist) * speed
            self.z += (dz / dist) * speed
        
        # จำกัดขอบเขต
        self.x = max(0.05, min(0.95, self.x))
        self.z = max(0.1, min(0.95, self.z))
        
        if self.shoot_cooldown > 0:
            self.shoot_cooldown -= 1
    
    def can_shoot(self):
        return self.has_gun and self.shoot_cooldown == 0
    
    def shoot(self, target):
        if self.can_shoot():
            self.shoot_cooldown = self.shoot_rate
            return Bullet(self.x, self.z, target)
        return None
    
    def draw(self):
        screen_x, screen_y, scale = world_to_screen(self.x, self.z)
        size = int(self.base_size * scale)
        
        if ai_hoshino_img:
            scaled_img = pygame.transform.scale(ai_hoshino_img, (size * 2, size * 2))
            img_rect = scaled_img.get_rect(center=(int(screen_x), int(screen_y)))
            screen.blit(scaled_img, img_rect)
        else:
            pygame.draw.circle(screen, self.color, (int(screen_x), int(screen_y)), size)
            if self.has_gun:
                gun_length = max(8, int(size * 1.2))
                pygame.draw.line(screen, WHITE, 
                               (int(screen_x), int(screen_y)), 
                               (int(screen_x), int(screen_y - gun_length)), 2)


# ============================================
# คลาส Crowd - กลุ่มคนทีมเรา
# ============================================
class Crowd:
    def __init__(self):
        self.people = [Person(0.5, 0.85, 1)]  # เริ่มที่กลางถนน ด้านหน้า
        self.center_x = 0.5  # 0.0 - 1.0
        self.center_z = 0.85  # อยู่ด้านหน้า
        self.count = 1
        
    def add_people(self, amount):
        if amount > 0:
            current = len(self.people)
            amount = min(amount, 500 - current)
            
            for _ in range(amount):
                angle = random.uniform(0, math.pi * 2)
                radius = random.uniform(0.02, 0.06)
                x = self.center_x + math.cos(angle) * radius
                z = self.center_z + math.sin(angle) * radius * 0.3
                self.people.append(Person(x, z, len(self.people) + 1))
            self.count = len(self.people)
            self.update_all_sizes()
    
    def remove_people(self, amount):
        if amount > 0 and len(self.people) > 0:
            amount = min(amount, len(self.people))
            self.people = self.people[:-amount]
            self.count = len(self.people)
            if self.count > 0:
                self.update_all_sizes()
    
    def update_all_sizes(self):
        for person in self.people:
            person.crowd_size = self.count
            person.update_size()
    
    def multiply_people(self, multiplier):
        if multiplier > 0:
            current = len(self.people)
            to_add = current * (multiplier - 1)
            to_add = min(to_add, 500 - current)
            if to_add > 0:
                self.add_people(to_add)
    
    def divide_people(self, divisor):
        if divisor > 0 and self.count > 0:
            new_count = max(1, self.count // divisor)
            to_remove = self.count - new_count
            if to_remove > 0:
                self.remove_people(to_remove)
    
    def power_people(self, power):
        current = len(self.people)
        if current > 0:
            new_count = min(500, int(current ** power))
            if new_count > current:
                self.add_people(new_count - current)
            elif new_count < current:
                self.remove_people(current - new_count)
    
    def sqrt_people(self):
        current = len(self.people)
        if current > 0:
            new_count = max(1, int(math.sqrt(current)))
            if new_count < current:
                self.remove_people(current - new_count)
    
    def move(self, dx):
        new_x = self.center_x + dx * 0.02
        self.center_x = max(0.1, min(0.9, new_x))
    
    def update(self):
        num = len(self.people)
        if num == 0:
            return
        
        max_per_ring = 12
        person_index = 0
        ring = 0
        
        while person_index < num:
            people_in_ring = min(max_per_ring + ring * 3, num - person_index)
            radius = 0.03 + ring * 0.04
            
            for i in range(people_in_ring):
                if person_index >= num:
                    break
                
                angle = (i / people_in_ring) * math.pi * 2 + (ring * 0.5)
                target_x = self.center_x + math.cos(angle) * radius
                target_z = self.center_z + math.sin(angle) * radius * 0.3
                
                self.people[person_index].update(target_x, target_z)
                person_index += 1
            
            ring += 1
    
    def try_shoot(self, enemies):
        bullets = []
        for person in self.people:
            closest_enemy = None
            min_dist = float('inf')
            
            for enemy in enemies:
                if not enemy.alive or not enemy.active:
                    continue
                dist = math.sqrt((enemy.x - person.x)**2 + (enemy.z - person.z)**2)
                if dist < min_dist and dist < 0.5:
                    min_dist = dist
                    closest_enemy = enemy
            
            if closest_enemy and person.can_shoot():
                bullet = person.shoot(closest_enemy)
                if bullet:
                    bullets.append(bullet)
        
        return bullets
    
    def draw(self):
        # เรียงตาม z เพื่อวาดจากไกลไปใกล้
        sorted_people = sorted(self.people, key=lambda p: p.z)
        for person in sorted_people:
            person.draw()


# ============================================
# คลาส Gate - ประตูเลือก
# ============================================
class Gate:
    def __init__(self, z, level):
        self.z = z  # ความลึก 0.0 - 1.0
        self.depth = 0.15
        self.used = False
        self.level = level
        
        # เลือก operation pool ตาม level
        if level == 1:
            pool = ops_level_1
        elif level == 2:
            pool = ops_level_2
        else:
            pool = ops_level_3
        
        # สุ่มเลือก operation สำหรับประตูซ้ายและขวา
        self.left_op = random.choice(pool)
        self.right_op = random.choice(pool)
    
    def check_collision(self, crowd):
        if self.used or len(crowd.people) == 0:
            return False
        
        if abs(crowd.people[0].z - self.z) < 0.08:
            if crowd.center_x < 0.5:  # ซ้าย
                self.left_op.apply(crowd)
                self.used = True
                return True
            else:  # ขวา
                self.right_op.apply(crowd)
                self.used = True
                return True
        return False
    
    def draw(self):
        if not self.used:
            gate_color = CYAN
            
            # วาดประตูซ้าย
            top_left_x, top_y, _ = world_to_screen(0.0, self.z - self.depth/2)
            top_mid_x, _, _ = world_to_screen(0.5, self.z - self.depth/2)
            bottom_left_x, bottom_y, _ = world_to_screen(0.0, self.z + self.depth/2)
            bottom_mid_x, _, _ = world_to_screen(0.5, self.z + self.depth/2)
            
            left_gate_points = [
                (top_left_x, top_y),
                (top_mid_x - 5, top_y),
                (bottom_mid_x - 5, bottom_y),
                (bottom_left_x, bottom_y)
            ]
            pygame.draw.polygon(screen, gate_color, left_gate_points)
            pygame.draw.polygon(screen, BLACK, left_gate_points, 5)
            
            # วาดข้อความซ้าย
            center_x = (top_left_x + top_mid_x + bottom_left_x + bottom_mid_x) / 4
            center_y = (top_y + bottom_y) / 2
            scale = 0.3 + 0.7 * self.z
            font_size = int(72 * scale)
            dynamic_font = pygame.font.Font(None, max(20, font_size))
            text = dynamic_font.render(self.left_op.symbol, True, WHITE)
            text_rect = text.get_rect(center=(int(center_x), int(center_y)))
            screen.blit(text, text_rect)
            
            # วาดประตูขวา
            top_right_x, _, _ = world_to_screen(1.0, self.z - self.depth/2)
            bottom_right_x, _, _ = world_to_screen(1.0, self.z + self.depth/2)
            
            right_gate_points = [
                (top_mid_x + 5, top_y),
                (top_right_x, top_y),
                (bottom_right_x, bottom_y),
                (bottom_mid_x + 5, bottom_y)
            ]
            pygame.draw.polygon(screen, gate_color, right_gate_points)
            pygame.draw.polygon(screen, BLACK, right_gate_points, 5)
            
            # วาดข้อความขวา
            center_x = (top_right_x + top_mid_x + bottom_right_x + bottom_mid_x) / 4
            text = dynamic_font.render(self.right_op.symbol, True, WHITE)
            text_rect = text.get_rect(center=(int(center_x), int(center_y)))
            screen.blit(text, text_rect)
            
            # วาดเส้นแบ่งกลาง
            pygame.draw.line(screen, WHITE, (top_mid_x, top_y), (bottom_mid_x, bottom_y), 8)


# ============================================
# คลาส Enemy - ศัตรู
# ============================================
class Enemy:
    def __init__(self, x, z):
        self.x = x  # 0.0 - 1.0
        self.z = z  # 0.0 - 1.0
        self.base_size = 10
        self.color = RED
        self.hp = 1
        self.speed = 0.006  # ลดความเร็วจาก 0.008 → 0.006
        self.alive = True
        self.active = False
        
    def activate(self):
        self.active = True
        
    def update(self, crowd):
        if not self.alive or not self.active:
            return
        
        if len(crowd.people) > 0:
            target = min(crowd.people, key=lambda p: 
                        math.sqrt((p.x - self.x)**2 + (p.z - self.z)**2))
            
            dx = target.x - self.x
            dz = target.z - self.z
            dist = math.sqrt(dx*dx + dz*dz)
            
            if dist > 0.01:
                self.x += (dx / dist) * self.speed
                self.z += (dz / dist) * self.speed
            
            if dist < 0.03:
                crowd.remove_people(1)
                self.alive = False
    
    def take_damage(self):
        self.hp -= 1
        if self.hp <= 0:
            self.alive = False
    
    def draw(self):
        if self.alive:
            screen_x, screen_y, scale = world_to_screen(self.x, self.z)
            size = int(self.base_size * scale)
            
            if knife_img:
                scaled_knife = pygame.transform.scale(knife_img, (size * 2, size * 2))
                knife_rect = scaled_knife.get_rect(center=(int(screen_x), int(screen_y)))
                screen.blit(scaled_knife, knife_rect)
            else:
                pygame.draw.circle(screen, self.color, (int(screen_x), int(screen_y)), size)
                pygame.draw.line(screen, WHITE, 
                               (int(screen_x - size), int(screen_y)), 
                               (int(screen_x + size), int(screen_y)), 2)


# ============================================
# คลาส Bullet - กระสุน
# ============================================
class Bullet:
    def __init__(self, x, z, target):
        self.x = x
        self.z = z
        self.speed = 0.03
        self.active = True
        self.target = target
        
    def update(self):
        if not self.active:
            return
            
        if not self.target or not self.target.alive:
            self.active = False
            return
        
        dx = self.target.x - self.x
        dz = self.target.z - self.z
        dist = math.sqrt(dx*dx + dz*dz)
        
        if dist > 0.01:
            self.x += (dx / dist) * self.speed
            self.z += (dz / dist) * self.speed
        
        if self.z < -0.1 or self.z > 1.1 or self.x < -0.1 or self.x > 1.1:
            self.active = False
    
    def check_hit(self, enemies):
        for enemy in enemies:
            if enemy.alive:
                dist = math.sqrt((enemy.x - self.x)**2 + (enemy.z - self.z)**2)
                if dist < 0.03:
                    enemy.take_damage()
                    self.active = False
                    return True
        return False
    
    def draw(self):
        if self.active:
            screen_x, screen_y, scale = world_to_screen(self.x, self.z)
            size = int(4 * scale)
            pygame.draw.circle(screen, YELLOW, (int(screen_x), int(screen_y)), max(2, size))


# ============================================
# คลาส Game
# ============================================
class Game:
    def __init__(self, level):
        self.level = level
        self.crowd = Crowd()
        self.gates = []
        self.enemies = []
        self.bullets = []
        self.lava_pits = []
        self.distance = 0
        self.spawn_timer = 0
        self.game_over = False
        self.won = False
        self.gates_passed = 0
        self.gates_needed = 4
        
        try:
            self.game_over_image = pygame.image.load('images.png')
            self.game_over_image = pygame.transform.scale(self.game_over_image, (400, 300))
        except:
            self.game_over_image = None
        
    def spawn_gate(self):
        self.gates.append(Gate(0.0, self.level))  # เริ่มที่ด้านหลัง
    
    def spawn_enemies(self):
        num_enemies = random.randint(8, 15)
        z = random.uniform(-0.2, 0.0)  # spawn ด้านหลัง
        for _ in range(num_enemies):
            x = random.uniform(0.1, 0.9)
            self.enemies.append(Enemy(x, z))
    
    def spawn_lava_pit(self):
        if self.level >= 2:
            x = random.uniform(0.2, 0.8)
            z = random.uniform(-0.3, 0.0)
            self.lava_pits.append(LavaPit(x, z))
    
    def start_game(self):
        """เริ่มเกมด้วย gate และ enemies ทันที"""
        self.spawn_gate()
        self.spawn_enemies()
        if self.level >= 2:
            self.spawn_lava_pit()
    
    def update(self):
        if self.game_over or self.won:
            return
        
        # เลื่อนวัตถุเข้ามาหาผู้เล่น (เพิ่ม z)
        # ใช้ perspective-corrected speed: ช้าตอนอยู่ไกล เร็วตอนอยู่ใกล้
        for gate in self.gates:
            perspective_factor = 0.3 + 0.7 * max(0, gate.z)  # 0.3-1.0 based on depth
            gate.z += SCROLL_SPEED * 0.003 * perspective_factor
        
        for enemy in self.enemies:
            perspective_factor = 0.3 + 0.7 * max(0, enemy.z)
            enemy.z += SCROLL_SPEED * 0.003 * perspective_factor
        
        for lava in self.lava_pits:
            perspective_factor = 0.3 + 0.7 * max(0, lava.z)
            lava.z += SCROLL_SPEED * 0.003 * perspective_factor
        
        self.spawn_timer += 1
        if self.spawn_timer % 220 == 0:  # ช้าลง: 180 → 220
            self.spawn_gate()
        if self.spawn_timer % 250 == 0:  # ช้าลง: 200 → 250
            self.spawn_enemies()
        if self.level >= 2 and self.spawn_timer % 350 == 0:  # ช้าลง: 280 → 350
            self.spawn_lava_pit()
        
        self.crowd.update()
        
        new_bullets = self.crowd.try_shoot(self.enemies)
        self.bullets.extend(new_bullets)
        
        for bullet in self.bullets:
            bullet.update()
            bullet.check_hit(self.enemies)
        
        for enemy in self.enemies:
            enemy.update(self.crowd)
        
        for lava in self.lava_pits:
            lava.check_collision(self.crowd)
        
        for gate in self.gates:
            if gate.check_collision(self.crowd):
                self.gates_passed += 1
                for enemy in self.enemies:
                    enemy.activate()
        
        # ลบวัตถุที่ผ่านไปแล้ว (z > 1.2)
        self.gates = [g for g in self.gates if g.z < 1.2]
        self.enemies = [e for e in self.enemies if e.alive and e.z < 1.2]
        self.bullets = [b for b in self.bullets if b.active]
        self.lava_pits = [l for l in self.lava_pits if l.active and l.z < 1.2]
        
        self.distance += SCROLL_SPEED
        
        if self.crowd.count <= 0:
            self.game_over = True
        
        if self.gates_passed >= self.gates_needed:
            self.won = True
    
    def draw(self):
        if stage_bg:
            screen.blit(stage_bg, (0, 0))
        else:
            screen.fill(BLACK)
        
        # วาดถนนแบบ 3D perspective
        road_points = [
            (ROAD_TOP_LEFT, ROAD_TOP_Y),      # บนซ้าย
            (ROAD_TOP_RIGHT, ROAD_TOP_Y),     # บนขวา
            (ROAD_BOTTOM_RIGHT, ROAD_BOTTOM_Y), # ล่างขวา
            (ROAD_BOTTOM_LEFT, ROAD_BOTTOM_Y)   # ล่างซ้าย
        ]
        pygame.draw.polygon(screen, GRAY, road_points)
        pygame.draw.polygon(screen, WHITE, road_points, 5)
        
        # วาดเส้นถนนตรงกลาง
        pygame.draw.line(screen, YELLOW, 
                        (WIDTH // 2, ROAD_TOP_Y), 
                        (WIDTH // 2, ROAD_BOTTOM_Y), 3)
        
        # วาดเส้นขอบถนนเพิ่มเติม
        for i in range(5):
            z = i * 0.25
            left, right = get_road_bounds(z)
            screen_y = ROAD_TOP_Y + (ROAD_BOTTOM_Y - ROAD_TOP_Y) * z
            pygame.draw.line(screen, (100, 100, 100), 
                           (int(left), int(screen_y)), 
                           (int(right), int(screen_y)), 1)
        
        # เรียง object ตาม z เพื่อวาดจากไกลไปใกล้
        all_objects = []
        
        for gate in self.gates:
            all_objects.append(('gate', gate, gate.z))
        
        for lava in self.lava_pits:
            all_objects.append(('lava', lava, lava.z))
        
        for enemy in self.enemies:
            all_objects.append(('enemy', enemy, enemy.z))
        
        for bullet in self.bullets:
            all_objects.append(('bullet', bullet, bullet.z))
        
        for person in self.crowd.people:
            all_objects.append(('person', person, person.z))
        
        # เรียงตาม z (ไกลไปใกล้)
        all_objects.sort(key=lambda x: x[2])
        
        # วาดทุก object
        for obj_type, obj, _ in all_objects:
            obj.draw()
        
        # HUD
        count_box = pygame.Rect(10, 10, 180, 80)
        pygame.draw.rect(screen, DARK_GRAY, count_box)
        pygame.draw.rect(screen, CYAN, count_box, 3)
        count_label = small_font.render("CROWD", True, CYAN)
        screen.blit(count_label, (20, 15))
        count_text = big_font.render(f"{self.crowd.count}", True, WHITE)
        screen.blit(count_text, (25, 40))
        
        level_box = pygame.Rect(WIDTH - 190, 10, 180, 80)
        pygame.draw.rect(screen, DARK_GRAY, level_box)
        pygame.draw.rect(screen, YELLOW, level_box, 3)
        level_label = small_font.render("LEVEL", True, YELLOW)
        screen.blit(level_label, (WIDTH - 175, 15))
        level_text = big_font.render(f"{self.level}", True, WHITE)
        screen.blit(level_text, (WIDTH - 155, 40))
        
        gates_box = pygame.Rect(10, 100, 180, 60)
        pygame.draw.rect(screen, DARK_GRAY, gates_box)
        pygame.draw.rect(screen, GREEN, gates_box, 3)
        gates_label = small_font.render("GATES", True, GREEN)
        screen.blit(gates_label, (20, 105))
        gates_text = font.render(f"{self.gates_passed}/{self.gates_needed}", True, WHITE)
        screen.blit(gates_text, (25, 125))
        
        if self.game_over:
            # แสดง ai_dead.png เต็มจอก่อน
            if ai_dead_img:
                screen.blit(ai_dead_img, (0, 0))
            else:
                overlay = pygame.Surface((WIDTH, HEIGHT))
                overlay.set_alpha(200)
                overlay.fill(BLACK)
                screen.blit(overlay, (0, 0))
            
            # ซ้อนทับด้วย images.png ตรงกลาง
            if self.game_over_image:
                img_x = WIDTH // 2 - 200
                img_y = HEIGHT // 2 - 150
                screen.blit(self.game_over_image, (img_x, img_y))
            else:
                text = big_font.render("GAME OVER!", True, RED)
                screen.blit(text, (WIDTH//2 - 200, HEIGHT//2 - 50))
            
            restart_btn = Button(WIDTH//2 - 150, HEIGHT//2 + 200, 300, 80, "RESTART", GREEN)
            restart_btn.draw()
            return restart_btn
        
        if self.won:
            if ai_win_img:
                screen.blit(ai_win_img, (0, 0))
            else:
                overlay = pygame.Surface((WIDTH, HEIGHT))
                overlay.set_alpha(180)
                overlay.fill(BLACK)
                screen.blit(overlay, (0, 0))
            
            info_box = pygame.Surface((500, 250))
            info_box.set_alpha(180)
            info_box.fill(BLACK)
            screen.blit(info_box, (WIDTH//2 - 250, HEIGHT//2 - 125))
            
            text = big_font.render("LEVEL COMPLETE!", True, GREEN)
            screen.blit(text, (WIDTH//2 - 280, HEIGHT//2 - 100))
            
            score_text = font.render(f"Survivors: {self.crowd.count}", True, WHITE)
            screen.blit(score_text, (WIDTH//2 - 130, HEIGHT//2 - 20))
            
            if self.level < 3:
                next_text = small_font.render("Press N for Next Level", True, GREEN)
                screen.blit(next_text, (WIDTH//2 - 135, HEIGHT//2 + 40))
            else:
                win_text = font.render("YOU WIN THE GAME!", True, YELLOW)
                screen.blit(win_text, (WIDTH//2 - 200, HEIGHT//2 + 40))
            
            menu_text = small_font.render("Press M for Menu", True, WHITE)
            screen.blit(menu_text, (WIDTH//2 - 100, HEIGHT//2 + 80))


# ============================================
# ฟังก์ชันหน้าเมนู
# ============================================
def draw_menu():
    if stage_bg:
        screen.blit(stage_bg, (0, 0))
    else:
        screen.fill(BLACK)
    
    # กล่องโปร่งแสงสำหรับ title
    title_box = pygame.Surface((700, 100))
    title_box.set_alpha(150)
    title_box.fill(BLACK)
    screen.blit(title_box, (WIDTH//2 - 350, 130))
    
    title = big_font.render("MATH CROWD RUNNER", True, CYAN)
    screen.blit(title, (WIDTH//2 - 320, 150))
    
    play_btn = Button(WIDTH//2 - 150, 300, 300, 100, "PLAY", GREEN)
    exit_btn = Button(WIDTH//2 - 150, 440, 300, 100, "EXIT", RED)
    
    play_btn.draw()
    exit_btn.draw()
    
    inst_font = pygame.font.Font(None, 24)
    inst1 = inst_font.render("Level 1: Basic Math (+, -, x)", True, BLACK)
    inst2 = inst_font.render("Level 2: Advanced (x, ÷, √, ^)", True, BLACK)
    inst3 = inst_font.render("Level 3: Extreme Difficulty", True, BLACK)
    
    screen.blit(inst1, (WIDTH//2 - 150, 560))
    screen.blit(inst2, (WIDTH//2 - 160, 590))
    screen.blit(inst3, (WIDTH//2 - 140, 620))
    
    return play_btn, exit_btn


# ============================================
# ลูปเกมหลัก
# ============================================
game = None
running = True

while running:
    mouse_pos = pygame.mouse.get_pos()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        if event.type == pygame.MOUSEBUTTONDOWN:
            if game_state == "MENU":
                play_btn, exit_btn = draw_menu()
                if play_btn.check_click(mouse_pos):
                    game_state = "PLAYING"
                    current_level = 1
                    game = Game(current_level)
                    game.start_game()  # เริ่มเกมทันที
                    # เล่นเพลง
                    try:
                        pygame.mixer.music.play(-1)  # -1 = loop ไม่รู้จบ
                        pygame.mixer.music.set_volume(0.5)
                    except:
                        pass
                elif exit_btn.check_click(mouse_pos):
                    running = False
            
            elif game_state == "PLAYING" and game:
                if game.game_over:
                    restart_btn = game.draw()
                    if restart_btn and restart_btn.check_click(mouse_pos):
                        current_level = 1
                        game = Game(current_level)
                        game.start_game()  # เริ่มเกมทันที
                        # เล่นเพลงใหม่
                        try:
                            pygame.mixer.music.play(-1)
                            pygame.mixer.music.set_volume(0.5)
                        except:
                            pass
        
        if event.type == pygame.KEYDOWN:
            if game_state == "PLAYING" and game:
                # เคลื่อนที่เฉพาะตอนกดปุ่ม (step-by-step)
                if event.key == pygame.K_LEFT or event.key == pygame.K_a:
                    game.crowd.move(-PLAYER_SPEED)
                elif event.key == pygame.K_RIGHT or event.key == pygame.K_d:
                    game.crowd.move(PLAYER_SPEED)
                # เมนูและการเปลี่ยนด่าน
                elif event.key == pygame.K_m:
                    game_state = "MENU"
                    game = None
                    # หยุดเพลงเมื่อกลับเมนู
                    try:
                        pygame.mixer.music.stop()
                    except:
                        pass
                elif event.key == pygame.K_n and game.won and current_level < 3:
                    current_level += 1
                    game = Game(current_level)
                    game.start_game()  # เริ่มด่านใหม่ทันที
    
    if game_state == "MENU":
        play_btn, exit_btn = draw_menu()
        play_btn.check_click(mouse_pos)
        exit_btn.check_click(mouse_pos)
        
    elif game_state == "PLAYING":
        if game:
            game.update()
            restart_btn = game.draw()
            
            if restart_btn and game.game_over:
                restart_btn.check_click(mouse_pos)
    
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
